In [1]:
from question import query_answer
from question import search
import pandas as pd

In [2]:
question_data = pd.read_csv('question_test_data.csv')

In [3]:
import pandas as pd
results = []
for i, question in enumerate(question_data['question']):
    result = query_answer(question, i)
    results += result
data = pd.DataFrame(results)
data

,question_id,question,rank,id,article,score
0,0,"Who is the author of the memoir ""Nicotine""?",1,17552,"The best cigarette you will ever smoke, Gregor...",3.518432
1,0,"Who is the author of the memoir ""Nicotine""?",2,17552,while smoking and dreamt of cigarettes of rela...,-2.276561
2,0,"Who is the author of the memoir ""Nicotine""?",3,17689,American novelists have long complained about ...,-6.128816
3,1,What skills are inmates learning in the innova...,1,17382,"LECCE, Italy One of his first students was a y...",5.459594
4,1,What skills are inmates learning in the innova...,2,17382,"Do remember, even if you had Trump over for di...",-2.192561
5,1,What skills are inmates learning in the innova...,3,17382,He now produces 16 different wines from grapes...,-4.931535
6,2,Who surrendered to the authorities for changin...,1,17547,A man who admitted changing the Hollywood sign...,8.200672
7,2,Who surrendered to the authorities for changin...,2,17832,"Grace Huezo, 20, a student at Hunter College, ...",-9.181833
8,2,Who surrendered to the authorities for changin...,3,17623,"Levi Felix, who wanted people to untangle them...",-9.527172
9,3,What is the name of the journalist and archivi...,1,17778,"the agency (the magician, who someone claimed ...",-1.510211


In [4]:
data = pd.DataFrame(data)
data.to_csv('questions_ranking.csv')

In [5]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('msmarco-distilbert-base-dot-prod-v3')

import faiss
index = faiss.read_index('data_article.index')

In [6]:
def mrr_score(answers, queries):
    '''answers is a list of list of ids'''
    score = []
    for i, answer in enumerate(answers):
        for j, index in enumerate(answer):
            if index == queries[i]:
                score.append(1 / (j + 1))
                break
        if len(score) < i + 1:
            score.append(0)
    return sum(score) / len(score) if len(score) > 0 else 0

In [7]:
answer_ids = []
for i, question in enumerate(question_data['question']):
    answer = search(question_data['question'][i], top_k=20, index=index, model=model)
    answers = [answer[x]['id'] // 10 for x in range(len(answer))]
    answer_ids.append(answers)
answer_ids

[[17552,
  17552,
  17542,
  17592,
  17699,
  17542,
  18332,
  18023,
  18163,
  17327,
  17960,
  18039,
  17733,
  17689,
  18163,
  17749,
  17541,
  17327,
  17494,
  17633],
 [17382,
  17382,
  17635,
  17382,
  17493,
  18176,
  18119,
  17820,
  17434,
  17686,
  17284,
  18255,
  18428,
  17612,
  17430,
  17639,
  18104,
  18176,
  18283,
  17749],
 [17547,
  17421,
  17633,
  17623,
  18111,
  18035,
  18159,
  18035,
  17832,
  17830,
  17749,
  17888,
  17832,
  17840,
  18165,
  18456,
  18300,
  18087,
  18240,
  17663],
 [18329,
  17594,
  17778,
  18329,
  18335,
  17327,
  18275,
  17388,
  18272,
  17726,
  18236,
  17635,
  18131,
  17562,
  18387,
  18446,
  17566,
  17411,
  17542,
  18428],
 [17727,
  17643,
  18254,
  17840,
  18210,
  17502,
  18456,
  18255,
  17785,
  17832,
  18023,
  18254,
  17765,
  18445,
  17727,
  17841,
  17693,
  17888,
  17727,
  18263],
 [18228,
  17384,
  18210,
  17770,
  17858,
  17670,
  18071,
  17885,
  18228,
  18118,
  181

In [8]:
mrr_score(answer_ids, question_data['doc_id'])

0.7935606060606062

In [9]:
rerank_data = pd.read_csv('questions_ranking.csv')
question_article_ids = {}
for i, question_id in enumerate(rerank_data['question_id']):
    if question_id not in question_article_ids:
        question_article_ids[question_id] = [rerank_data['id'][i]]
    else:
        question_article_ids[question_id].append(rerank_data['id'][i])

rerank_article_ids = [question_article_ids[x] for x in question_article_ids]
rerank_article_ids

[[17552, 17552, 17689],
 [17382, 17382, 17382],
 [17547, 17832, 17623],
 [17778, 17562, 18428],
 [17841, 18254, 18254],
 [18228, 18228, 17384],
 [18443, 18364, 17333],
 [18170, 18170, 17940],
 [17980, 18068, 18391],
 [17838, 17977, 17838],
 [17544, 17544, 18079]]

In [10]:
mrr_score(rerank_article_ids, question_data['doc_id'])

1.0

In [11]:
def accuracy_score(answers, queries):
    '''answers is a list of list of ids'''
    score = []
    for i, answer in enumerate(answers):
        for index in answer:
            if index == queries[i]:
                score.append(1)
                break
        if len(score) != i + 1:
            score.append(0)
            
    return sum(score) / len(score) if len(score) > 0 else 0

In [12]:
accuracy_score(answer_ids, question_data['doc_id'])

1.0

In [13]:
print('Accuracy search:', accuracy_score(answer_ids, question_data['doc_id']))
print('MRR search:', mrr_score(answer_ids, question_data['doc_id']))
print('MRR rerank:', mrr_score(rerank_article_ids, question_data['doc_id']))

Accuracy search: 1.0
MRR search: 0.7935606060606062
MRR rerank: 1.0
